In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
import math
import numpy as np

class Frequency():

    SAMPLING_RATE = 10 * 44100 # Hz https://en.wikipedia.org/wiki/MP3
    DURATION = .20  # .20 second clip
    X = np.arange(0, DURATION, 1.00/SAMPLING_RATE)
    FUNC = np.sin

    def __init__(self, frequency):
        self.frequency = frequency
        
    @property
    def wave(self):
        y = self.FUNC(self.frequency * self.X * 2 * math.pi)  # 1 hz * freq
        return y

class Note(Frequency):
    
    NOTES = "C C# D D# E F F# G G# A A# B".split()
    TWELFTH_ROOT_OF_TWO = math.pow(2, 1.00/12.00)
    # Harmonic ratios for each of the intervals
    JUST_INTONATION_RATIOS = [
        1, 16/15.0, 9/8.0, 6/5.0, 5/4.0, 4/3.0, 45/32.0,
        3/2.0, 8/5.0, 5/3.0, 16/9.0, 15/8.0
    ]

    JUST_INTONATION_LCMS = [
        1, 30, 16, 10, 8, 6, 64,
        4, 10, 3, 18, 16
    ]
    
    def __init__(self, name = "A", octave = 4):
        assert(name in self.NOTES)
        self.name = name
        self.octave = int(octave)
        
    @property
    def frequency_justly_tuned(self):
        # ("C", 4): 261.63
        difference = Note(self.name, 4).ordinal - Note("C", 4).ordinal
        freq = 261.63 * self.JUST_INTONATION_RATIOS[difference]
        difference_2 = self.ordinal - Note(self.name, 4).ordinal
        if difference_2 >= 0:
            freq *= 2.0 ** difference_2
        else:
            freq /= (0.5) ** (-1 * difference_2)
        return freq
        
    @property
    def frequency_equal_temprament(self):
        difference = self.ordinal - (Note("A", 4).ordinal)
        return 440.00 * (self.TWELFTH_ROOT_OF_TWO ** difference)

    @property
    def stable_length(self):
        difference = self.ordinal - (Note("C", 4).ordinal)
        return self.JUST_INTONATION_LCMS[difference]

    @property
    def frequency(self):
        return self.frequency_justly_tuned
    
    @property
    def ordinal(self):
        return self.octave * 12 + self.NOTES.index(self.name)
        
    def __repr__(self):
        return "{}{} ({:.2f} Hz)".format(
            self.name, self.octave, self.frequency)

In [4]:
def f(theta, a, b):
    x = np.cos((b/a) * theta) * np.cos(theta)
    y = np.cos((b/a) * theta) * np.sin(theta)
    
    #distances = np.square(x[10:] - x[0]) + np.square(y[10:] - y[0])
    #print(np.argmin(distances) + 10)
    
    x = list(reversed(x))
    y = list(reversed(y))
    return {'x': x, 'y': y, 'c': (theta)}

def plot_curves(notes, tuning):
    plt.figure(figsize=(20,20))
    for i, n in enumerate(notes):
        length = 2 * Note(n).stable_length
        x = np.arange(0, length * np.pi, np.pi / 1000.0)
        ax = plt.subplot(2,2,i+1)
        ax.set_facecolor("#000000")
        if(tuning == 'equal_temprament'):
            plt.scatter(s=1, **f(x, Note('C').frequency_equal_temprament, Note(n).frequency_equal_temprament))
        elif(tuning == 'justly_tuned'):
            plt.scatter(s=1, **f(x, Note('C').frequency_justly_tuned, Note(n).frequency_justly_tuned))
        else:
            plt.scatter(s=1, **f(x, Note('C').frequency, Note(n).frequency))


In [5]:
# for n in sorted([Note(name) for name in Note.NOTES], key=lambda x: x.stable_length):
#     print(n, n.stable_length)

```
C: Perfect Unison
F: Perfect 4th
G: Perfect 5th

D: Major 2nd
E: Major 3rd
A: Major 6th
B: Major 7th
```

```
C#: Minor 2nd
D#: Minor 3rd
G#: Minor 6th
A# / Bb: Minor 7th

F#: Tritone
```

In [6]:
plot_curves(list('FG'), 'justly_tuned')

In [7]:
plot_curves(list('FG'), 'equal_temprament')

In [8]:
plot_curves(['D', 'E', 'A', 'B'], 'justly_tuned')

In [9]:
plot_curves(['D', 'E', 'A', 'B'], 'equal_temprament')

In [10]:
plot_curves(['C#', 'D#', 'G#', 'A#'], 'justly_tuned')

In [11]:
plot_curves(['C#', 'D#', 'G#', 'A#'], 'equal_temprament')

In [12]:
plot_curves(['F#', 'F#'], 'justly_tuned')

In [13]:
plot_curves(['F#', 'F#'], 'equal_temprament')